## SWaT Dataset → Siemens PLC DB Converter

Part of the research toolset for:

**_“Towards AI-Based Anomaly Detection at the Edge:  
Evaluating Real-Time Cyber Defense in Programmable Logic Controllers”_**

---

This tool converts labeled SWaT dataset `.csv` files into  
**Siemens-compatible `DATA_BLOCK` (`.db`) files** for PLC testing and simulation.

Each row is validated and normalized into `REAL`-typed values,  
including input features and a binary label (`normal = 0.0`, `attack = 1.0`).

The generated DB file contains:

- A 2D `ARRAY[0..N, 0..M] OF REAL` structure  
- Fully deterministic format, ready for Siemens TIA Portal  
- Suitable for both testing and educational use

---

## User Guide

This tool runs fully offline. Configuration is done via constants at the top of the script.

---

### Step 1: File Setup

```python
SWAT_INPUT_DATASET_FILE_NAME = "your_file.csv"
SWAT_OUTPUT_DB_FILE_NAME     = "your_file.db"
```

You may also change:
- `BLOCK_NAME`, `VERSION`, or `VAR_NAME` if needed

---

### Step 2: Run the Notebook

Run **manually** (not with “Run All”). The converter will:

- Validate the CSV structure  
- Normalize numeric inputs  
- Map labels (`normal` or `attack`)  
- Generate the `.db` output

---

### Step 3: Review Output

After execution:

- Output appears in the defined folder  
- Overwrite must be confirmed if file exists  
- You get summary statistics:

```text
[INFO] Total samples: 11545
[INFO] Total columns: 41 (40 features + 1 label)
[INFO] Label distribution:
   - 0.0: 9471 samples (82.03%)
   - 1.0: 2074 samples (17.97%)
[INFO] Integer-to-float conversions: 3832
```

---

### Notes

- Unknown labels or column mismatch will raise errors  
- Integers are always converted (e.g., `250` → `250.0`)  
- All output values are `REAL` (float) for PLC compatibility

---

## Legal Notice  

- **Author:** Anonymous  
- **Website:** ---  
- **Contact:** later  
- **License:** Creative Commons BY-NC 4.0  
- **Version:** v1.0.1  
- **Copyright:** © 2025–2026

This software is intended exclusively for **educational and research purposes**, or other **non-commercial applications**.  
Use in **industrial production environments** is prohibited.  
The **author must be credited** in all derivative or redistributive works.  
**Commercial use** requires **explicit written permission** from the author.  


In [ ]:
from pathlib import Path
import re
import csv
import os
import ipywidgets as widgets
from IPython.display import display
BASE_DIR = Path.cwd()
PROJECT_DIR = BASE_DIR.parent

In [16]:
# --- Input CSV folder
SWAT_DATASET_DIR = PROJECT_DIR / "dataset"

# --- Output DB folder
OUTPUT_DB_DIR = PROJECT_DIR / "dataset/db"


# --- List contents of input and output folders
print(f"[INFO] Input folder ({SWAT_DATASET_DIR}):")
for f in sorted(SWAT_DATASET_DIR.glob("*")):
    print(f"   - {f.name}")

print(f"[INFO] Output folder ({OUTPUT_DB_DIR}):")
for f in sorted(OUTPUT_DB_DIR.glob("*")):
    print(f"   - {f.name}")


[INFO] Input folder (c:\Repos\Towards-AI-Based-Anomaly-Detection-at-the-Edge-Evaluating-Real-Time-CyberDefense-in-PLC\dataset):
   - Attack_v0_first_attack.csv
   - db
   - SWaT_Dataset_Normal_v0.csv
[INFO] Output folder (c:\Repos\Towards-AI-Based-Anomaly-Detection-at-the-Edge-Evaluating-Real-Time-CyberDefense-in-PLC\dataset\db):
   - Attack_v0_first_attack.db


In [19]:
# --- Input CSV filename (source dataset)           
SWAT_INPUT_DATASET_FILE_NAME = "Attack_v0_first_attack.csv"

# --- Output DB filename (Siemens PLC format) 
SWAT_OUTPUT_DB_FILE_NAME     = "Attack_v0_first_attack.db"

# ----- DB INTERNAL CONTENT  -----
BLOCK_NAME   = "Attack_v0_first_attack"   # name of the DATA_BLOCK
VERSION      = "0.1"                      # DB version
VAR_NAME     = "DATA"                     # name of the 2D array

REQUIRED_FEATURE_COUNT = 40 # Number of input features expected per row (excluding the target label)

# --- Warn if output file already exists
if (OUTPUT_DB_DIR / SWAT_OUTPUT_DB_FILE_NAME).exists():
    print(f"[WARNING] Output DB file already exists: {OUTPUT_DB_DIR / SWAT_OUTPUT_DB_FILE_NAME}")
else:
    print(f"[INFO] Output DB file will be created: {OUTPUT_DB_DIR / SWAT_OUTPUT_DB_FILE_NAME}")


[INFO] Output DB file will be created: c:\Repos\Towards-AI-Based-Anomaly-Detection-at-the-Edge-Evaluating-Real-Time-CyberDefense-in-PLC\dataset\db\Attack_v0_first_attack.db


In [20]:
# ----- FILE PATHS -----
IN_FILE  = SWAT_DATASET_DIR / SWAT_INPUT_DATASET_FILE_NAME
OUT_FILE = OUTPUT_DB_DIR / SWAT_OUTPUT_DB_FILE_NAME

# ----- DB DIMENSIONS -----
DATA_COLS = REQUIRED_FEATURE_COUNT          # number of data columns
LABEL_COL_INDEX = REQUIRED_FEATURE_COUNT    # label column index
TOTAL_COLS = REQUIRED_FEATURE_COUNT + 1     # total columns stored
       
# ----- DB INDEX OFFSETS -----       
ROW_START = 0   # starting row index in DB
COL_START = 0   # starting column index in DB

# ----- CSV INPUT CONFIG -----
CSV_DELIMITER        = ","          # delimiter of the CSV input file 
CSV_ENCODING         = "utf-8"      # encoding of the CSV input file
SKIP_HEADER_LINES    = 1            # number of header lines to skip in the CSV input file

# ----- STATS -----
INTEGER_CONVERSION_COUNT = 0

# ----- REGEXES -----
_integer_re = re.compile(r"^[+-]?\d+$")  
_numeric_point_re = re.compile(r"^[+-]?(?:\d+(?:\.\d+)?|\.\d+)(?:[eE][+-]?\d+)?$")

In [21]:
# --- Normalize numeric input: trim, convert integers to float format ---
def normalize_data_cell(cell: str) -> str:
    global INTEGER_CONVERSION_COUNT
    
    c = cell.strip().strip(",;")
    
    if _integer_re.fullmatch(c):
        INTEGER_CONVERSION_COUNT += 1 
        c = f"{c}.0"

    return c


# --- Map raw label values to standard numeric form ("normal" → 0.0, "attack" → 1.0) ---
def map_label_cell(cell: str) -> str:
    raw = cell.strip().strip("\"'")
    if _numeric_point_re.fullmatch(raw):
        return raw
    
    lowered = raw.casefold().strip()
    
    if lowered == "normal":
        return "0.0"
    
    if lowered == "attack":
        return "1.0"
    
    raise ValueError(f"Unknown label value: '{cell}'")


# --- Read CSV rows: enforce column count, normalize values, and map labels ---
def read_rows(path: Path) -> list[list[str]]:
    rows: list[list[str]] = []
    delimiter = "\t" if CSV_DELIMITER in {"\\t", "\t"} else CSV_DELIMITER

    with path.open("r", encoding=CSV_ENCODING, errors="ignore", newline="") as f:
        reader = csv.reader(f, delimiter=delimiter)

        for _ in range(SKIP_HEADER_LINES):
            next(reader, None)

        for line_idx, cols in enumerate(reader, start=1 + SKIP_HEADER_LINES):
            if not cols or all(c.strip() == "" for c in cols):
                continue

            if len(cols) < TOTAL_COLS:
                raise ValueError(
                    f"{line_idx}. row: too few columns ({len(cols)} < {TOTAL_COLS})"
                )

            if len(cols) > TOTAL_COLS:
                raise ValueError(
                    f"{line_idx}. row: too many columns ({len(cols)} > {TOTAL_COLS})"
                )

            try:
                data_cells = [normalize_data_cell(c) for c in cols[:DATA_COLS]]
                label_cell = map_label_cell(cols[LABEL_COL_INDEX])
            except ValueError as e:
                raise ValueError(f"Error in row {line_idx}: {e}") from None

            rows.append(data_cells + [label_cell])

    if not rows:
        raise ValueError("No valid data rows found after header.")
    return rows


# --- Generate PLC-compatible DB text from normalized data rows ---
def build_db_text(rows: list[list[str]]) -> str:
    n_rows = len(rows)
    row_end = ROW_START + n_rows - 1
    col_end = COL_START + TOTAL_COLS - 1  # 0..40

    header = [
        f'DATA_BLOCK "{BLOCK_NAME}"',
        "{ S7_Optimized_Access := 'TRUE' }",
        f"VERSION : {VERSION}",
        "NON_RETAIN",
        "   VAR",
        f"      {VAR_NAME} : ARRAY[{ROW_START}..{row_end}, {COL_START}..{col_end}] OF REAL;",
        "   END_VAR",
        "BEGIN",
    ]

    body: list[str] = []
    for i, row in enumerate(rows):
        for j, val in enumerate(row):
            if val == "":
                continue
            body.append(f"{VAR_NAME}[{i+ROW_START},{j+COL_START}] := {val};")

    footer = ["END_DATA_BLOCK"]
    return "\n".join(header + body + footer) + "\n"



In [22]:
def main():
    rows = read_rows(IN_FILE)
    db_text = build_db_text(rows)

    # ---- Basic statistics ----
    num_rows = len(rows)
    num_columns = TOTAL_COLS
    label_values = [row[-1] for row in rows]

    from collections import Counter
    label_counts = Counter(label_values)
    label_stats = {label: f"{(count / num_rows) * 100:.2f}%" for label, count in label_counts.items()}

    print(f"[INFO] Total samples: {num_rows}")
    print(f"[INFO] Total columns: {num_columns} ({DATA_COLS} features + 1 label)")
    print(f"[INFO] Label distribution:")
    for label, count in label_counts.items():
        print(f"       - {label}: {count} samples ({label_stats[label]})")
    print(f"[INFO] Integer-to-float conversions (e.g. 250 → 250.0): {INTEGER_CONVERSION_COUNT}")


    # ---- Output handling ----
    out = widgets.Output()

    if os.path.exists(OUT_FILE):
        print(f"[WARNING] File already exists at: {OUT_FILE}")
        print("Press YES to overwrite or NO to skip saving.")

        yes_button = widgets.Button(description="Yes", button_style="danger")
        no_button  = widgets.Button(description="No",  button_style="success")

        def _disable_buttons():
            yes_button.disabled = True
            no_button.disabled  = True

        def on_yes_clicked(b):
            _disable_buttons()
            with out:
                out.clear_output()
                OUT_FILE.write_text(db_text, encoding=CSV_ENCODING)
                print(f"[INFO] DB file overwritten: {OUT_FILE}")

        def on_no_clicked(b):
            _disable_buttons()
            with out:
                out.clear_output()
                print("[INFO] DB file exists, not saved.")

        yes_button.on_click(on_yes_clicked)
        no_button.on_click(on_no_clicked)

        display(widgets.HBox([yes_button, no_button]))
        display(out)

    else:
        OUT_FILE.write_text(db_text, encoding=CSV_ENCODING)
        print(f"[INFO] DB file created at: {OUT_FILE}")

In [23]:
if __name__ == "__main__":
    main()

[INFO] Total samples: 2694
[INFO] Total columns: 41 (40 features + 1 label)
[INFO] Label distribution:
       - 0.0: 1754 samples (65.11%)
       - 1.0: 940 samples (34.89%)
[INFO] Integer-to-float conversions (e.g. 250 → 250.0): 42347
[INFO] DB file created at: c:\Repos\Towards-AI-Based-Anomaly-Detection-at-the-Edge-Evaluating-Real-Time-CyberDefense-in-PLC\dataset\db\Attack_v0_first_attack.db
